In [1]:
pwd

'D:\\Thesis Work\\InitialDemoWork\\Cambridge_bigger USING LIBROSA'

### Imports

In [17]:
#kernel restarted for sleep, resume the thresholding work

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm_notebook

import sklearn.metrics as sklm
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from keras import models
from keras.models import Model, load_model
from keras import callbacks
import keras
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose, AveragePooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import SGD, Adam

### Load Phone List

In [16]:
phones = np.load("phoneList.npy",allow_pickle=True)
phones = phones[(phones != 'sil') & (phones != 'sp') & (phones != 'spn') & (phones != 'ZH')]
len(phones)

38

In [15]:
print("Count of Phones in L2 Dev Corpus (Correct + Substitution Errors):")
total = 0
for phone in phones:
    if phone!= "ZH":
        count = np.load("./L2 Phones Dev/"+phone+"_y.npy").shape[0]
        print(phone+": ",count)
        total+=count
print('Total: ', total)

Count of Phones in L2 Dev Corpus (Correct + Substitution Errors):
AA:  450
R:  1340
EY:  610
B:  418
IY:  1250
AH:  2904
AE:  810
K:  920
S:  1304
F:  813
T:  1591
L:  981
OW:  638
N:  1946
D:  1284
M:  971
SH:  398
W:  889
EH:  689
V:  615
IH:  2092
ER:  742
AY:  683
HH:  819
AO:  504
NG:  418
JH:  159
Z:  1046
UW:  701
G:  240
AW:  194
P:  554
TH:  244
Y:  386
CH:  240
UH:  160
OY:  60
DH:  963
Total:  31026


In [13]:
print("Count of Phones in L2 Test Corpus (Correct + Substitution Errors):")
total = 0
for phone in phones:
    count = np.load("./L2 Phones Test/"+phone+"_y.npy").shape[0]
    print(phone+": ",count)
    total+=count
print('Total: ', total)

Count of Phones in L2 Test Corpus (Correct + Substitution Errors):
AA:  757
R:  2586
EY:  1337
B:  1249
IY:  2869
AH:  6036
AE:  1906
K:  1633
S:  2554
F:  1327
T:  3976
L:  2832
OW:  919
N:  4131
D:  2926
M:  1977
SH:  715
W:  1751
EH:  1920
V:  1384
IH:  4171
ER:  1891
AY:  1165
HH:  1952
AO:  1013
NG:  701
JH:  439
Z:  2172
UW:  1049
G:  696
AW:  498
ZH:  107
P:  1332
TH:  486
Y:  436
CH:  450
UH:  439
OY:  116
DH:  2055
Total:  65953


In [11]:
print("Count of Phones in L2 Corpus (Correct + Substitution Errors):")
total = 0
for phone in phones:
    if phone!="ZH":
        count = np.load("./L2 Phones Test/"+phone+"_y.npy").shape[0]+np.load("./L2 Phones Dev/"+phone+"_y.npy").shape[0]
        print(phone+": ",count)
        total+=count
print('Total: ', total)

Count of Phones in L2 Corpus (Correct + Substitution Errors):
AA:  1207
R:  3926
EY:  1947
B:  1667
IY:  4119
AH:  8940
AE:  2716
K:  2553
S:  3858
F:  2140
T:  5567
L:  3813
OW:  1557
N:  6077
D:  4210
M:  2948
SH:  1113
W:  2640
EH:  2609
V:  1999
IH:  6263
ER:  2633
AY:  1848
HH:  2771
AO:  1517
NG:  1119
JH:  598
Z:  3218
UW:  1750
G:  936
AW:  692
P:  1886
TH:  730
Y:  822
CH:  690
UH:  599
OY:  176
DH:  3018
Total:  96872


## Thresholding & Evaluation

### X_Validation + Thresholding

In [18]:
thresholdList = []
for phone in tqdm_notebook(phones):
    print("\n\n------->>>>>>Phone: ", phone)
    print()
    X_val = np.load("./L2 Phones Dev/"+phone+"_X.npy",allow_pickle=True)
    y_val = np.load("./L2 Phones Dev/"+phone+"_y.npy",allow_pickle=True)
    print("Total Samples validation X_val: ", X_val.shape[0])
    print("Shape of y_val (only correct yet): ", y_val.shape)
    
    print("\n-----------MIN-MAX NORMALISATION-----------\n")
    
    Xmax = np.load("Cambridge_bigger Train MinMax/max_"+phone+".npy", allow_pickle=True)
    Xmin = np.load("Cambridge_bigger Train MinMax/min_"+phone+".npy", allow_pickle=True)
    X_val = (X_val - Xmin) / (Xmax - Xmin)
    print("X_val shape: ", X_val.shape)
    
    print("\n-----------THRESHOLDING-----------\n")
    
    autoencoder = load_model("./Models/"+phone+".h5")
    
    validationSetLength = X_val.shape[0]
    accs = []
    f1mis = []

    y_val_mses = np.array([autoencoder.evaluate(X_val[i:i+1],X_val[i:i+1],verbose=0) for i in range(validationSetLength)])
    
    print("Total Thresholds: ",y_val_mses.size)
    
    for threshold in sorted(y_val_mses): #ek ek kar k put in +ve
        y_pred = 1*(y_val_mses <= threshold)

        accs.append(sklm.accuracy_score(y_val,y_pred))
        f1mis.append(sklm.precision_recall_fscore_support(y_val,y_pred)[2][0])

    print("Max Achievable Accuracy (Overall): ", max(accs))
    print("Max Achievable Accuracy (Overall) on Threshold: ", np.argmax(accs))

    print("Max Achievable F-1 score Threshold: ", sorted(y_val_mses)[np.argmax(f1mis)])
    print("Max Achievable Accuracy On This Threshold: ", accs[np.argmax(f1mis)])
    print("Max Achievable F-1 score for Mispronunciations: ", max(f1mis))
    print("============================================")
    thresholdList.append(sorted(y_val_mses)[np.argmax(f1mis)])



------->>>>>>Phone:  AA

Total Samples validation X_val:  450
Shape of y_val (only correct yet):  (450,)

-----------MIN-MAX NORMALISATION-----------

X_val shape:  (450, 13, 50, 3)

-----------THRESHOLDING-----------

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Total Thresholds:  450


D:\Anaconda3\envs\thesisWork\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Max Achievable Accuracy (Overall):  0.8355555555555556
Max Achievable Accuracy (Overall) on Threshold:  448
Max Achievable F-1 score Threshold:  7.693684165133163e-05
Max Achievable Accuracy On This Threshold:  0.17333333333333334
Max Achievable F-1 score for Mispronunciations:  0.28735632183908044


------->>>>>>Phone:  R

Total Samples validation X_val:  1340
Shape of y_val (only correct yet):  (1340,)

-----------MIN-MAX NORMALISATION-----------

X_val shape:  (1340, 13, 50, 3)

-----------THRESHOLDING-----------

Total Thresholds:  1340
Max Achievable Accuracy (Overall):  0.9164179104477612
Max Achievable Accuracy (Overall) on Threshold:  1339
Max Achievable F-1 score Threshold:  5.928109385422431e-05
Max Achievable Accuracy On This Threshold:  0.09253731343283582
Max Achievable F-1 score for Mispronunciations:  0.15555555555555556


------->>>>>>Phone:  EY

Total Samples validation X_val:  610
Shape of y_val (only correct yet):  (610,)

-----------MIN-MAX NORMALISATION-----------



Total Thresholds:  1284
Max Achievable Accuracy (Overall):  0.92601246105919
Max Achievable Accuracy (Overall) on Threshold:  1283
Max Achievable F-1 score Threshold:  0.00015467428602278233
Max Achievable Accuracy On This Threshold:  0.866822429906542
Max Achievable F-1 score for Mispronunciations:  0.25327510917030566


------->>>>>>Phone:  M

Total Samples validation X_val:  971
Shape of y_val (only correct yet):  (971,)

-----------MIN-MAX NORMALISATION-----------

X_val shape:  (971, 13, 50, 3)

-----------THRESHOLDING-----------

Total Thresholds:  971
Max Achievable Accuracy (Overall):  0.9897013388259527
Max Achievable Accuracy (Overall) on Threshold:  970
Max Achievable F-1 score Threshold:  0.00012447641347534955
Max Achievable Accuracy On This Threshold:  0.41709577754891863
Max Achievable F-1 score for Mispronunciations:  0.027491408934707903


------->>>>>>Phone:  SH

Total Samples validation X_val:  398
Shape of y_val (only correct yet):  (398,)

-----------MIN-MAX NORMA

X_val shape:  (701, 13, 50, 3)

-----------THRESHOLDING-----------

Total Thresholds:  701
Max Achievable Accuracy (Overall):  0.9557774607703281
Max Achievable Accuracy (Overall) on Threshold:  696
Max Achievable F-1 score Threshold:  0.0006616336759179831
Max Achievable Accuracy On This Threshold:  0.9557774607703281
Max Achievable F-1 score for Mispronunciations:  0.1142857142857143


------->>>>>>Phone:  G

Total Samples validation X_val:  240
Shape of y_val (only correct yet):  (240,)

-----------MIN-MAX NORMALISATION-----------

X_val shape:  (240, 13, 50, 3)

-----------THRESHOLDING-----------

Total Thresholds:  240
Max Achievable Accuracy (Overall):  0.9625
Max Achievable Accuracy (Overall) on Threshold:  239
Max Achievable F-1 score Threshold:  0.0001949677534867078
Max Achievable Accuracy On This Threshold:  0.8875
Max Achievable F-1 score for Mispronunciations:  0.2285714285714286


------->>>>>>Phone:  AW

Total Samples validation X_val:  194
Shape of y_val (only correct y

In [19]:
thresholdList

[7.693684165133163e-05,
 5.928109385422431e-05,
 0.000299896695651114,
 0.0001756464916979894,
 0.00028882193146273494,
 5.4747535614296794e-05,
 0.00016961117216851562,
 7.041809294605628e-05,
 0.0004905149107798934,
 0.0001256945397472009,
 9.105699427891523e-05,
 0.00027136903372593224,
 0.00010336262494092807,
 0.00011225897469557822,
 0.00015467428602278233,
 0.00012447641347534955,
 0.000540056440513581,
 0.00015675551549065858,
 7.470211130566895e-05,
 0.00018472567899152637,
 0.0001927341363625601,
 9.811010386329144e-05,
 0.00017400234355591238,
 0.00045656634029001,
 0.00022717384854331613,
 0.0052166166715323925,
 0.00039807925350032747,
 0.00021537832799367607,
 0.0006616336759179831,
 0.0001949677534867078,
 0.00025385720073245466,
 9.692783351056278e-05,
 0.03754192218184471,
 0.0008086412562988698,
 0.0001943986426340416,
 0.00042681029299274087,
 0.0002892148622777313,
 0.0001501347724115476]

### Thresholding

## Testing

### X_test + Predictions

In [20]:
f1ScoresList = []
for t,phone in enumerate(tqdm_notebook(phones)):
    print("\n\n------->>>>>>Phone: ", phone)
    print()
    X_test = np.load("./L2 Phones Test/"+phone+"_X.npy",allow_pickle=True)
    y_test = np.load("./L2 Phones Test/"+phone+"_y.npy",allow_pickle=True)

    print("Total Samples test X_test: ", X_test.shape[0])
    print("Shape of y_test (only correct yet): ", y_test.shape)

    print("\n-----------MIN-MAX NORMALISATION-----------\n")
    
    Xmax = np.load("Cambridge_bigger Train MinMax/max_"+phone+".npy",allow_pickle=True)
    Xmin = np.load("Cambridge_bigger Train MinMax/min_"+phone+".npy",allow_pickle=True)
    X_test = (X_test - Xmin) / (Xmax - Xmin)
    print("X_test Shape: ",X_test.shape)
    
    print("\n-----------PREDICTING-----------\n")
    
    autoencoder = load_model("./Models/"+phone+".h5")
    
    y_pred = [1 if autoencoder.evaluate(X_test[i:i+1],X_test[i:i+1],verbose=0)<=thresholdList[t]\
     else 0 for i in range(X_test.shape[0])]
    
    print("Phone: ",phone)
    print(classification_report(y_test, y_pred, target_names=['Mispronounced','Correct']))
    print(confusion_matrix(y_test, y_pred, labels=range(2)))

    print(sklm.roc_curve(y_test,y_pred))
    print('FAR = ', sklm.roc_curve(y_test,y_pred)[0][1]*100,'%')
    print('FRR = ', (1 - sklm.roc_curve(y_test,y_pred)[1][1])*100,'%')
    print("AUC_SCORE = ", sklm.roc_auc_score(y_test,y_pred)*100,'%')
    print("============================================")
    f1ScoresList.append(sklm.precision_recall_fscore_support(y_test,y_pred)[2][0])



------->>>>>>Phone:  AA

Total Samples test X_test:  757
Shape of y_test (only correct yet):  (757,)

-----------MIN-MAX NORMALISATION-----------

X_test Shape:  (757, 13, 50, 3)

-----------PREDICTING-----------

Phone:  AA
               precision    recall  f1-score   support

Mispronounced       0.14      0.96      0.25       109
      Correct       0.71      0.02      0.03       648

    micro avg       0.15      0.15      0.15       757
    macro avg       0.43      0.49      0.14       757
 weighted avg       0.63      0.15      0.06       757

[[105   4]
 [638  10]]
(array([0.        , 0.03669725, 1.        ]), array([0.       , 0.0154321, 1.       ]), array([2, 1, 0]))
FAR =  3.669724770642202 %
FRR =  98.4567901234568 %
AUC_SCORE =  48.936742552950506 %


------->>>>>>Phone:  R

Total Samples test X_test:  2586
Shape of y_test (only correct yet):  (2586,)

-----------MIN-MAX NORMALISATION-----------

X_test Shape:  (2586, 13, 50, 3)

-----------PREDICTING-----------

Phone:



------->>>>>>Phone:  T

Total Samples test X_test:  3976
Shape of y_test (only correct yet):  (3976,)

-----------MIN-MAX NORMALISATION-----------

X_test Shape:  (3976, 13, 50, 3)

-----------PREDICTING-----------

Phone:  T
               precision    recall  f1-score   support

Mispronounced       0.06      0.88      0.12       266
      Correct       0.91      0.09      0.16      3710

    micro avg       0.14      0.14      0.14      3976
    macro avg       0.49      0.48      0.14      3976
 weighted avg       0.85      0.14      0.16      3976

[[ 234   32]
 [3385  325]]
(array([0.        , 0.12030075, 1.        ]), array([0.        , 0.08760108, 1.        ]), array([2, 1, 0]))
FAR =  12.030075187969924 %
FRR =  91.2398921832884 %
AUC_SCORE =  48.365016314370834 %


------->>>>>>Phone:  L

Total Samples test X_test:  2832
Shape of y_test (only correct yet):  (2832,)

-----------MIN-MAX NORMALISATION-----------

X_test Shape:  (2832, 13, 50, 3)

-----------PREDICTING----------

FRR =  26.70863309352518 %
AUC_SCORE =  58.88833051206094 %


------->>>>>>Phone:  IH

Total Samples test X_test:  4171
Shape of y_test (only correct yet):  (4171,)

-----------MIN-MAX NORMALISATION-----------

X_test Shape:  (4171, 13, 50, 3)

-----------PREDICTING-----------

Phone:  IH


D:\Anaconda3\envs\thesisWork\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


               precision    recall  f1-score   support

Mispronounced       0.13      1.00      0.24       557
      Correct       0.00      0.00      0.00      3614

    micro avg       0.13      0.13      0.13      4171
    macro avg       0.07      0.50      0.12      4171
 weighted avg       0.02      0.13      0.03      4171

[[ 557    0]
 [3614    0]]
(array([0., 1.]), array([0., 1.]), array([1, 0]))
FAR =  100.0 %
FRR =  0.0 %
AUC_SCORE =  50.0 %


D:\Anaconda3\envs\thesisWork\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)




------->>>>>>Phone:  ER

Total Samples test X_test:  1891
Shape of y_test (only correct yet):  (1891,)

-----------MIN-MAX NORMALISATION-----------

X_test Shape:  (1891, 13, 50, 3)

-----------PREDICTING-----------

Phone:  ER
               precision    recall  f1-score   support

Mispronounced       0.21      0.93      0.34       395
      Correct       0.82      0.08      0.15      1496

    micro avg       0.26      0.26      0.26      1891
    macro avg       0.51      0.51      0.25      1891
 weighted avg       0.69      0.26      0.19      1891

[[ 368   27]
 [1376  120]]
(array([0.        , 0.06835443, 1.        ]), array([0.       , 0.0802139, 1.       ]), array([2, 1, 0]))
FAR =  6.8354430379746836 %
FRR =  91.97860962566845 %
AUC_SCORE =  50.59297366817843 %


------->>>>>>Phone:  AY

Total Samples test X_test:  1165
Shape of y_test (only correct yet):  (1165,)

-----------MIN-MAX NORMALISATION-----------

X_test Shape:  (1165, 13, 50, 3)

-----------PREDICTING----------

AUC_SCORE =  57.996351474612354 %


------->>>>>>Phone:  P

Total Samples test X_test:  1332
Shape of y_test (only correct yet):  (1332,)

-----------MIN-MAX NORMALISATION-----------

X_test Shape:  (1332, 13, 50, 3)

-----------PREDICTING-----------

Phone:  P
               precision    recall  f1-score   support

Mispronounced       0.12      1.00      0.22       165
      Correct       1.00      0.00      0.00      1167

    micro avg       0.12      0.12      0.12      1332
    macro avg       0.56      0.50      0.11      1332
 weighted avg       0.89      0.12      0.03      1332

[[ 165    0]
 [1166    1]]
(array([0., 0., 1.]), array([0.00000000e+00, 8.56898029e-04, 1.00000000e+00]), array([2, 1, 0]))
FAR =  0.0 %
FRR =  99.91431019708654 %
AUC_SCORE =  50.04284490145673 %


------->>>>>>Phone:  TH

Total Samples test X_test:  486
Shape of y_test (only correct yet):  (486,)

-----------MIN-MAX NORMALISATION-----------

X_test Shape:  (486, 13, 50, 3)

-----------PREDICTING-----

In [21]:
f1ScoresList

[0.24647887323943662,
 0.21147826086956525,
 0.27314112291350534,
 0.15189873417721517,
 0.10149253731343284,
 0.14000532339632685,
 0.19652551574375676,
 0.07456404088995791,
 0.0,
 0.024042073628850486,
 0.12046332046332046,
 0.129153605015674,
 0.40469738030713637,
 0.10883439704400404,
 0.21444201312910285,
 0.02484472049689441,
 0.09090909090909091,
 0.1011169900058789,
 0.19821059876118377,
 0.3507246376811594,
 0.2356175972927242,
 0.34408602150537637,
 0.11285266457680251,
 0.0892857142857143,
 0.11518324607329841,
 0.11139240506329115,
 0.4444444444444445,
 0.6970684039087948,
 0.03773584905660377,
 0.23529411764705882,
 0.2766798418972332,
 0.22058823529411764,
 0.5945945945945946,
 0.0,
 0.03532008830022075,
 0.18439716312056736,
 0.04395604395604396,
 0.6741116751269036]

In [22]:
print("Mean F1 Score: ", round(np.mean(f1ScoresList),2)*100,"%")

Mean F1 Score:  20.0 %
